# 作業3 : 利用 RL 逃離迷宮
> **說明**

Using Q-learining or any reinforcement learning algorithm to find out the way to escape the maze.

請利用 Q-learning 或任何你知道的RL方法去逃離迷宮

請隨意修改這個 colab 的程式來完成訓練

數字僅供繪製地圖時參考用

迷宮本身為 21 * 11 的迷宮

起點左上角 (0,0) 終點為右下角 (20,10)

X 表示牆壁位置(顏色為綠色方便區分) O 表示寶藏位置(顏色為橘色方便區分) S 表示起點位置 G 表示終點位置 黃色底色表示可能路徑 請利用以下程式碼將地圖設定完成

移動時不可穿過障礙物

注意事項:

訓練次數（MAX_EPISODES 參數）不得超過 1000 次。

以能到終點為主要作業目標 步數越少分數越高。

請自行將規則補在程式內，並制定良好的 Reward 讓你更快走完。

寶藏計分方式為參數 Score，每踩到一個寶箱 Score +1，每個寶箱只能獲得一次分數，每個 episode 需重置，且每回合都必須計算 Score 分數，最高 5 分。

文件內請附上你認為最好的一筆訓練結果（步數 + 寶藏），並將結果保存於 colab 輸出結果中。計分將以這筆訓練結果作為依據！

In [16]:
import numpy as np
import pandas as pd
import time

pd.set_option('display.max_rows', 300)

"""設定 x, y 軸長度，

範例為 40 * 30 的無障礙迷宮

並設定可行動動作為上下左右

目標位置在最右下角

以及各項參數設定

SCORE 為計算寶藏分數用

得分計算方法請自行補上
"""

# todo: 修改迷宮大小
from enum import Enum
WALLS = [4, 5, 7, 9, 22, 23, 25, 30, 31, 35, 39, 43, 45, 47, 49, 50, 51,
      53, 55, 57, 58, 59, 61, 65, 71, 74, 80, 85, 88, 90, 94, 97, 100,
      101, 102, 104, 109, 110, 111, 113, 114, 119, 120, 127, 128, 129, 132,
      134, 136, 141, 142, 143, 145, 151, 153, 155, 157, 158, 164, 166, 169,
      172, 176, 178, 181, 183, 186, 187, 190, 191, 193, 195, 196, 206, 211, 214, 226, 229]
N_STATES_x = 21
N_STATES_y = 11
GOAL = 230
START = 0
TREASURES = [6, 79, 170, 212, 227]

ACTIONS = ["left", "right", "up", "down"]

EPSILON = 0.9
ALPHA = 0.05
GAMMA = 0.9
MAX_EPISODES = 500
FRESH_TIME = 0

SCORE = 0

get_treasures = []
# class ACTIONS(Enum):
#   LEFT = -1
#   RIGHT = 1
#   UP = -N_STATES_x
#   DOWN = N_STATES_x

In [17]:
"""建立一開始的空Q table"""

def build_q_table(N_STATES_x, N_STATES_y, actions):
    table = pd.DataFrame(
        np.zeros((N_STATES_x * N_STATES_y, len(actions))),
        columns=actions,
    )
    return table

In [18]:
"""如何選擇動作"""

def choose_action(state, q_table):

    # 取table內index從state開始之後的資料
    state_actions = q_table.iloc[state, :]

    # 最一開始或是epsilon>0.9，隨機挑選一個action
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):
        action_name = np.random.choice(ACTIONS)

    # idxmax()回傳每一row中數值最大的index
    else:
        action_name = state_actions.idxmax()
    return action_name

In [19]:
"""Reward 設定

(以向右走為例 剩餘動作請自行補上)
"""

REWARD_TREASURE = 3
REWARD_GOAL = 5
REWARD_FORWARD = -2
REWARD_BAD = -3

def get_next_state(state, action):
  next_state = state
  if action == "right":
    next_state += 1
  elif action == "left":
    next_state += -1
  elif action == "up":
    next_state += -N_STATES_x
  elif action == "down":
    next_state += N_STATES_x
  if next_state in WALLS or \
     next_state < 0 or \
     next_state > 230 :
    return [state, False]
  elif next_state % 21 == 0 and \
       action == "right":
    return [state, False]
  elif (next_state + 1) % 21 == 0 and \
       action == "left":
    return [state, False]
  elif next_state == GOAL:
    return [next_state, True]
  else:
    return [next_state, False]

def get_env_feedback(state, action, get_treasures):
#   print(f"now:{state}")
#   print(f"action:{action}")
  next_state, result = get_next_state(state, action)
  if next_state == state:
    reward = REWARD_BAD
  elif result:
    reward = REWARD_GOAL
  elif next_state in TREASURES and next_state not in get_treasures:
    reward = REWARD_TREASURE
    get_treasures.append(next_state)
    # TREASURES.remove(next_state)
    # print("get treasure!!!!!")
  else:
    reward = REWARD_FORWARD
  return next_state, reward, get_treasures

In [21]:
"""利用選擇的動作更新畫面
可自行選擇是否印出
"""

def update_env(state, episode, step_counter):

    # 環境地圖初始化 "-"代表空
    env_map = [["-" for i in range(N_STATES_x)] for j in range(N_STATES_y)]
    # T代表目標位置
    env_map[int(GOAL / N_STATES_x)][GOAL % N_STATES_x] = "T"
    # 顯示牆壁
    for wall in WALLS:
      env_map[int(wall / N_STATES_x)][wall % N_STATES_x] = "X"
    for treasure in TREASURES:
      env_map[int(treasure / N_STATES_x)][treasure % N_STATES_x] = "$$"
    result = []
    # 如果走到終點表示當前的回合已結束
    if state == 230:
        # 紀錄回合數、步數
        interaction = f"Episode:{episode + 1} total_steps:{step_counter}"
        # interaction = "Episode %s: total_steps=%s" % (episode + 1, step_counter, )
        result.append(interaction)
        print(result)
        print("\r", end="")

        # print("=============================")
    # 如果不是走到終點的話，標記目前的位置為 "o"
    else:
      env_map[int(state / N_STATES_x)][int(state % N_STATES_x)] = "o"
      # time.sleep(FRESH_TIME)


In [22]:
"""# 更新 Q table"""
def rl():
    # 創建Q table
    q_table = build_q_table(N_STATES_x, N_STATES_y, ACTIONS)
    for episode in range(MAX_EPISODES):
        # TREASURES = [6, 79, 170, 212, 227]
        step_counter = 0
        S = 0
        is_terminated = False
        path = []
        get_treasures = []
        update_env(S, episode, step_counter)
        while not is_terminated:
            A = choose_action(S, q_table)
            path.append(S)
            S_, R, get_treasures = get_env_feedback(S, A, get_treasures)
            # 從 q table 中獲取當前狀態和預測行動的Q值
            q_predict = q_table.loc[S, A]
            # 如果未到達終點
            if S_ != 230:
                # 計算目前的Q值：當前的 reward + 下一個 state 可以得到的最大Q值
                q_target = R + GAMMA * q_table.iloc[S_, :].max()
            # 到達終點
            else:
                # 目標Q值為當前 reward
                q_target = R
                is_terminated = True
            # 更新 Q table
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)
            S = S_
            update_env(S, episode, step_counter + 1)
            step_counter += 1
        if step_counter < 400 and len(get_treasures) > 3:
            print(f"路線{path}")
            print(f"獲得寶藏{get_treasures}")
            print(f"分數:{len(get_treasures)}")
            time.sleep(2)
    return all, q_table

In [23]:
"""執行程式"""

if __name__ == "__main__":
    q_table = rl()
    print("\r\nQ-table:\n")
    print(q_table)

['Episode:1 total_steps:3181']
['Episode:2 total_steps:5573']
['Episode:3 total_steps:4956']
['Episode:4 total_steps:4582']
['Episode:5 total_steps:3501']
['Episode:6 total_steps:2901']
['Episode:7 total_steps:2788']
['Episode:8 total_steps:2996']
['Episode:9 total_steps:1798']
['Episode:10 total_steps:2683']
['Episode:11 total_steps:2319']
['Episode:12 total_steps:2595']
['Episode:13 total_steps:1700']
['Episode:14 total_steps:2032']
['Episode:15 total_steps:2220']
['Episode:16 total_steps:2025']
['Episode:17 total_steps:1859']
['Episode:18 total_steps:1742']
['Episode:19 total_steps:1465']
['Episode:20 total_steps:1570']
['Episode:21 total_steps:1822']
['Episode:22 total_steps:1483']
['Episode:23 total_steps:1537']
['Episode:24 total_steps:1637']
['Episode:25 total_steps:1301']
['Episode:26 total_steps:1539']
['Episode:27 total_steps:1434']
['Episode:28 total_steps:1283']
['Episode:29 total_steps:1462']
['Episode:30 total_steps:1341']
['Episode:31 total_steps:1220']
['Episode:32 tota